In [293]:
import requests
from parsel import Selector
from collections import OrderedDict
url = 'http://139.59.208.213/Z6JHLZCM/pelennor/'

In [294]:
def get_odds(r):
    sel = Selector(text=r.text)
    extr = sel.css('table td::text').extract()
    #print (extr)
    indexes = []
    id_names = sel.css('input::attr("id")').extract()
    for id_val in sel.css('input::attr("id")').extract():
        #indexes.append(int((id_val[3:4] + id_val[5:6])))
        indexes.append(id_val)
    ps = {}
    for idx in range(0, len(indexes)):
        if extr[idx*5+3] not in ps:
            ps[extr[idx*5+3]] = {}
        ps[extr[idx*5+3]].update({indexes[idx]: extr[idx*5: idx*5+3]})
    #ps = list(map(float, ps))
    return ps


In [295]:
total = 3.0
r = requests.get(url)
odds = get_odds(r)
print(odds)

{'Club Vales of Anduin v Cuivienen FC': {'id_2-14-money': ['2.71', '2.74', '2.63'], 'id_5-14-money': ['2.71', '2.69', '2.69'], 'id_3-14-money': ['2.44', '3.19', '2.57']}, 'FC Dark Land v VfL Dale Dragonslayers': {'id_1-12-money': ['3.33', '2.36', '2.57'], 'id_4-12-money': ['3.29', '2.73', '2.26'], 'id_6-12-money': ['3.34', '2.47', '2.44']}, 'SSC Dunharrow v AS Carn Dum': {'id_3-8-money': ['3.12', '2.65', '2.41'], 'id_5-8-money': ['3.15', '2.6', '2.43'], 'id_1-8-money': ['3.76', '2.36', '2.36']}, 'Bree City FC v Hobbiton CF': {'id_0-3-money': ['3.42', '1.84', '3.62'], 'id_1-3-money': ['3.45', '1.85', '3.55'], 'id_6-3-money': ['3.05', '2.05', '3.36']}, 'FC Fornost v PFC Avallone': {'id_1-0-money': ['3.32', '2.26', '2.7'], 'id_5-0-money': ['3.48', '2.07', '2.92'], 'id_3-0-money': ['3.71', '2.1', '2.72']}, 'PSV Harlond v PFC CSKA Egladil': {'id_3-15-money': ['2.48', '2.95', '2.69'], 'id_2-15-money': ['2.52', '3.2', '2.47'], 'id_4-15-money': ['2.27', '3.49', '2.58']}, 'Rivendell FC v SL Ang

In [296]:
def do_bet():
    global odds, total
    import math
    
    all_ids = set()
    
    for match, match_data in odds.items():
        for bookie, bookie_data in match_data.items():
            all_ids.add(bookie)

    for match, match_data in odds.items():
        #print(match)

        data = OrderedDict()
        for id_ in all_ids:
            data['{}-winner'.format(id_[3:-6])] = "1"
            data['{}-money'.format(id_[3:-6])] = "0.00"

        match_ids = []
        odds_for = {
            '1': [],
            'X': [],
            '2': [],
        }
        
        for bookie, bookie_data in match_data.items():
            match_ids.append(bookie)
            odds_for['1'].append(float(bookie_data[0]))
            odds_for['X'].append(float(bookie_data[1]))
            odds_for['2'].append(float(bookie_data[2]))
    
        if len(match_ids) < 3:
            continue
    
        #print(odds_for)

        bets = {}
        while odds_for:
            best_odds = 0.
            best_result = None
            for result, bet_odds in odds_for.items():
                m = max(bet_odds)
                if m > best_odds:
                    best_odds = m
                    best_result = result
            index = odds_for[best_result].index(best_odds)
            del odds_for[best_result]
            for bet_odds in odds_for.values():
                bet_odds[index:index + 1] = [-1]
            bets[best_result] = (index, best_odds)
        #print(repr(bets))

        total_odds = sum(map(lambda b: b[1], bets.values()))
        a1 = round(total / bets['1'][1], 2)
        a2 = round(total / bets['2'][1], 2)
        aX = round(total / bets['X'][1], 2)

        required = a1 + a2 + aX
        #print(a1, a2, aX, required, total, required < total)

        if total >= required:
            place_bets = {}
            for result, (index, odds_) in bets.items():
                if result == '1':
                    ammount = a1
                elif result == '2':
                    ammount = a2
                else:
                    ammount = aX
                place_bets[index] = (result, ammount, odds_)

            #print(place_bets)
            if all(p[1] * p[2] > required for p in place_bets.values()):
                for i, pb in place_bets.items():
                    data['{}-winner'.format(match_ids[i][3:-6])] = pb[0]
                    data['{}-money'.format(match_ids[i][3:-6])] = pb[1]
                #print('betting', data)
                break

    #print(data)
    if True:
        r = requests.post(url, data=data)
        #print(r.text)
        sel = Selector(text=r.text)
        total = float(sel.css('.money::text').extract_first())
        extr = sel.css('table td::text').extract()
        #print (total)
        #for i in range(0, len(extr), 8):
        #    print(extr[i:i + 8])
        r = requests.post(url)
        odds = get_odds(r)
        #print(odds)


In [297]:
while total < 2000:
    do_bet()
    print (total)

1693.42
1693.42
1771.53
1847.29
1847.29
1931.94


TypeError: float() argument must be a string or a number, not 'NoneType'

In [279]:
sel = Selector(text=r.text)
extr = sel.css('table td::text').extract()
ps = []
ps = extr[0:3] + extr[5:8] + extr[10:13]
ps = list(map(float, ps))
ps

[2.74, 4.0, 1.99, 3.06, 4.0, 1.86, 2.97, 14.0, 1.42]